In [1]:
%env CUDA_VISIBLE_DEVICES=2

import rupo.api
import tensorflow as tf

sess = tf.InteractiveSession()

env: CUDA_VISIBLE_DEVICES=2


Using TensorFlow backend.


In [2]:
MODEL_PATH_BASE = '../models/subtitles_songs_poetry_with_meter_shakespeare_02_05_2019_500k_iters'

RUPO_DATA_ROOT = '../data/rupo/'
RUPO_STRESS_MODEL_PATH   = RUPO_DATA_ROOT + \
                           'stress_models/stress_ru_LSTM64_dropout0.2_acc99_wer8.h5'
RUPO_ZALYZNIAK_DICT_PATH = RUPO_DATA_ROOT + 'dict/zaliznyak.txt'
del RUPO_DATA_ROOT

In [3]:
def process_en_poem(poem):
    return poem.strip().split('\n')

In [4]:
import os
import sys
sys.path.append(os.path.abspath('../code'))

from rhyme_translator import RhymeType, RhymeTranslator, set_rhyme_debug_print, set_meter_debug_print
from translation_model import TranslationModel

In [5]:
def make_rupo_engine():
    rupo_engine = rupo.api.Engine(language = 'ru')
    rupo_engine.load(stress_model_path = RUPO_STRESS_MODEL_PATH,
                     zalyzniak_dict = RUPO_ZALYZNIAK_DICT_PATH)
    return rupo_engine

global_rupo_engine = make_rupo_engine()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model1 = TranslationModel(filename = MODEL_PATH_BASE + '.pkl')
model2 = TranslationModel(filename = MODEL_PATH_BASE + '_50k_fine_tune.pkl')
model3 = TranslationModel(filename = MODEL_PATH_BASE + '_100k_fine_tune.pkl')

In [7]:
translator1 = RhymeTranslator(model1, global_rupo_engine)
translator2 = RhymeTranslator(model2, global_rupo_engine)
translator3 = RhymeTranslator(model3, global_rupo_engine)
translators = [translator1, translator2, translator3]

In [8]:
def translate_poem(poem, **kwargs):
    
    lines = process_en_poem(poem)
    
    for i, translator in enumerate(translators):
        print('Model {}'.format(i + 1))
        translations = translator.translate_lines(lines, **kwargs)
        translations = [t if l.strip() != '' else '' for t, l in zip(translations, lines)]
        print('\n'.join(translations))
        print()

def translate_poem_with_rhyme(poem, **kwargs):
    
    if 'rhyme_type' not in kwargs:
        print('--- Rhyme type: SUFFIX ---\n')
        translate_poem_with_rhyme(poem, rhyme_type = RhymeType.SUFFIX, **kwargs)
        print('\n--- Rhyme type: WORD ---\n')
        translate_poem_with_rhyme(poem, rhyme_type = RhymeType.WORD, **kwargs)
    
    lines = process_en_poem(poem)
    
    for i, translator in enumerate(translators):
        print('Model {}'.format(i + 1))
        
        curr_lines = []
        for line in lines:
            if line.strip() == '':
                translations = translator.translate_lines_with_rhyme(curr_lines, **kwargs)
                print('\n'.join(translations))
                print('---')
                curr_lines = []
            else:
                curr_lines.append(line)
        if curr_lines:
            translations = translator.translate_lines_with_rhyme(curr_lines, **kwargs)
            print('\n'.join(translations))
        print()

In [34]:
poem = '''
Let this wind blow through the night
And we are going away
The animal hasn't crossed the street
Because it was too tired
'''

translate_poem(poem, sample_temperature = 0)
print()
print('sample_temperature = 0.3')
print()
translate_poem(poem, sample_temperature = 0.3)
print()
print('sample_temperature = 0.5')
print()
translate_poem(poem, sample_temperature = 0.5)

Model 1
позволь этот ветер взорвать всю ночь
и мы уходим отсюда
зверь не выбежал на улицу
слишком устал

Model 2
пусть этот ветер дует сквозь ночь
и мы уходим
животное не пересек дорогу
потому что слишком устала

Model 3
пусть дует ветер в эту ночь
и улетаем мы прочь
животное не мигала не по улице
потому что было слишком устало


sample_temperature = 0.3

Model 1
пусть будет ветер во всю ночь
и мы уходим отсюда
животных не пошел через улицу
я просто слишком устал

Model 2
пусть этот ветер дует сквозь ночь
и мы тоскуем
животное не пересек через дорогу
потому что слишком устала

Model 3
пусть этот ветер дует в эту ночь
и мы идем прочь
животное не подставлял ту дорогу
потому что было слишком устало


sample_temperature = 0.5

Model 1
дай эти ветра взорвать всю ночь
и мы покинем
животное не вылезла на улицу
потому что он слишком устал

Model 2
пусть этот ветер дует сквозь ночь
и мы уходим
животное не пересек улицу
потому что слишком устала

Model 3
пусть этот ветер дует в эту ночь
и воскли

In [22]:
# "The Road Not Taken" by Robert Frost, published in 1916

poem = '''
Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day!
Yet knowing how way leads on to way,
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I—
I took the one less traveled by,
And that has made all the difference.
'''

translate_poem(poem)

Model 1
два пути ложились в желтые дрова
и то и другое
и быть однажды путником был я
и выглядел так как я мог
туда где он прогибается в кушетке

выбор был другим как и честно
может быть гораздо лучше
потому что это была трава и хотела одеть
но несмотря за то что это там
я носил их над одним и тем же

и у вас обоих
не прохожий был темнокожий
я был первым в другой день
знать какой путь в пути
я сомневалась если я должна вернуться

я скажу это вдох
еще раз и века и отсюда
две дороги направлялись в дрова и я
я проходил мимо
и всё изменилось

Model 2
две дороги поднимались в желтом лесу
и прости что я еще и не вместе
и одним путником я стоял
и взглянула вниз вниз так как я
туда где это скрипело под взлётом

потом взяла друг от друга что же справедливо
может быть и получше
ведь дело было в траву и хотел носить
туда туда туда
я носил их точно так же

и та же обоим обоим в постель
никто не был черным
хранил я в другой день
еще не зная как отправиться в свой путь в путь
смогу ли я когда-нибудь 

In [30]:
translate_poem(poem,
               sample_temperature = 0.3,
               stresses_attempts = 10)

Model 1
направлялись в желтый вуд
но я мог не поехать
и однажды путником был я
и выглядел так как я могла
куда он колебался в звонке

тогда был другой как и честно
может быть лучше
потому что это была трава и хотел одеть
но за этим там
носил их над одним и тем же

и одинаково одинаково
на пути но дождь был черный
я был первым в другой день
знать какой путь в пути
я сомневалась если я должен вернуться

я скажу это вдох
вне века и века отсюда
две дороги направлялись в лес и я
я не был один
все изменилось

Model 2
две дороги сложены в желтом лесу
но и не я
и одним путником я стоял
и оглянулась так как я
туда где билась под рассветом

потом в другой так же как справедливый
и может быть лучше и получше
ведь дело было в траву и хотел носить
хотя и туда туда туда
я носил их точно так же

и в то утро приходят в постель
ни что ни была ни черна ни дня
о когда жил я в другой день
дорогу в свой путь в путь
смогу ли я когда-нибудь вернуться

я вздыхаю этот вздох
долгие дни и века
две дороги сложены

In [60]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
два пути ложились в морозном месте
но я мог бы и не вместе
и быть одним путником и давно я была
и выглядел так как я могла
туда где он прогибается в кушетке
---
так же как и всё по-честному
и то что это к лучшему
потому что это была трава и хотела одеть
но несмотря за то что это там
я носил их над одним и тем же
---
и у вас обоих
никаких шагов у прохожих чернокожих
я был первый раз и навсегда
знать какой путь на пути сюда
я сомневалась если я должна вернуться
---
я скажу это вдох
еще раз и века и отсюда
двух дорог ворвались в дрова и меня
ты напал на меня
и всё изменилось

Model 2
две дороги поднимались в пустынном золоте
и прости что я еще и не вместе
и одним путником был я
и взглянула вниз вниз так как я
туда где это скрипело под взлётом
---
так же как так же справедлива
возможно лучшие права
потому что было в траву и был носим
хотя за то что рядом с ним
я носил их точно так же
---
и одним утром
ни шаг не гиден был в черном
хранил я в другой день
еще не зная как отправиться в

In [65]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
ложились в желтые лески
но я не могу без вести
и быть одним путником и давно я была
и выглядел так как я могла
туда где он прогибается в кушетке
---
он был так же как и был честен
может быть на члене
потому что это была трава и хотела одеть
но несмотря за то что это там
я носил их над одним и тем же
---
и стало одинаково ложью
не прохожий был темнокожий
я сохранил первое место для другого
знать какой путь к дороге
я сомневалась если я должна вернуться
---
я со своим побалем
раз и века и дальше
у меня и в лес
я напал на след
и всё изменилось

Model 2
две дороги поднимались в желтом лесу
и прости что я еще и не вместе
и одним путником я оставался со мной
и взглянула вниз вниз так как я смог
туда где это скрипело под взлётом
---
потом взяла друг от друга что же справедливо
может быть и получше
ведь дело было в траву и хотел носить
туда туда туда
я носил их точно так же
---
и та же обоим обоим в постель
никто не был черным
о я остался один день за другой
еще не зная как я представл

In [66]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
две и встретились в морозном месте
но я бы не вместе
и быть одним путником давно я была
и выглядел так как я могла
и входила
---
и всё по-честному
и то что это к лучшему
все из-за травы и она не носила
хотя это может быть там
носил их над одним и тем же
---
и у вас обоих
нет дороги нет у чернокожих
я был первый раз и навсегда
знать как путь на пути сюда
я сомневалась если я должен вернуться
---
я строю этот вздох
на века и века отсюда
двух дорог ворвались в дрова и меня
ты напал на меня
все изменилось

Model 2
я оказался в огромном золоте
и прости я уже не вместе
и одним путником был я
я обернулась так как я
туда где расступилась под бомботом
---
так же как так же справедлива
возможно лучшие права
он был был и раньше был носим
хотя несмотря на то что рядом с ним
я носил их точно так же
---
и одним утром
с каждым шагом что гиден был в черном
я хранил за ещё один день
доведи свой путь в путь в путь
смогу ли я когда-нибудь вернуться
---
я буду говорить с этим дыханьем
многие века 

In [67]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
направлялись в желтые лески
простите я не могу без вести
давно я была
он выглядел так как я могла
туда где он сожрал под поднетом
---
ты был честен
может быть на члене
потому что это была трава и хотела одеть
и это там
я носил их вместе с одним и тем же
---
и одинаково ложью
нет но не прохожий был темнокожий
я хранил первое место для другого
знать какой путь к дороге
я сомневалась если мне вернуться
---
я со своим побалем
и века и дальше
и в лес
я напал на след
все изменилось

Model 2
две дороги сложены в желтом лесу
и прости я это не вместе
и одним путником я оставался со мной
и выглянула вниз вниз так как я смог
прямо под взлётом
---
и в тот же как справедливый
и вероятно лучше и получше
ведь дело было в траву и хотел носить
хотя это проходящий мимо
я носил их точно так же
---
утро и в одно утро
всё было не так как блэк
я хранил на один день за другой
пока еще не зная как путь с тобой
осмелилась ли я когда-нибудь
---
я это вдыхаем
века и века
я и в лесу
я получил меньше на до

In [11]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 100)

Model 1
мы ложились в морозном месте
извините но я не вместе
давно я была
выглядел такой как я могла
в сугробах
---
то был другим как и всё по-честному
может быть это к лучшему
из-за травы и то что ты носила
но несмотря за то что это там
глядя на них
---
и у вас обоих
среди чернокожих
я был первый раз и навсегда
знать как путь на пути сюда
я сомневалась если мне вернуться
---
я скажу им
вне века и века отсюда
они направлялись в дрова и меня
я плыл на меня
все изменилось

Model 2
я оказался в золоте
и прости что я и не вместе
и одним путником был я
и взглянула всё не так как я
прямо под пятном
---
потом понял что так же справедлива
и возможно лучшие права
я был носим
хотя это рядом с ним
мучили их одного и те же
---
и одним утром
был не в черном
я был за ещё один день
смотря в свой путь в путь
мне ли если когда-то вернуться
---
я буду говорить с этим дыханьем
несколько веков и со временем
я поднимались в древесину
отнял у меня меньше одну
всё изменилось

Model 3
двух дорог ружав на желт

In [12]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 100)

Model 1
ложились в желтые лески
простите я не могу без вести
давно я была
и выглядел так как я могла
туда попасть под скорбеть
---
тогда был так же как и был честен
кто может быть на члене
из-за травы и то что ты носила
но несмотря за то что это там
он был тем же
---
и все одинаково ложью
и не проезжий был темнокожий
у меня был первый день для другого
знать какой путь к дороге
я сомневалась если мне вернуться
---
я с побалем
раз и века и дальше
и в лес
я взял на след
все изменилось

Model 2
две дороги сложены в желтом лесу
и прости почему бы не вернуться
и одним путником я был со мной
и взглянула вниз вниз так как я смог
туда где он тянулся под копыт
---
потом понял что так же справедливо
возможно и получше получше
потому что хотела и носила
хотя тот путь туда туда
я далась в них и так же
---
и утро одно и тем же утром
не шатнулся истраден был черным
о в первый день за другой
пока не зная что есть путь с тобой
вряд ли мне вернуться
---
я вдыхаю
в это время и века
я и в лесу
я принял не

In [ ]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 1000)

In [ ]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 1000)

In [38]:
# "The Tyger" by William Blake, published in 1794

poem = '''
Tiger Tiger, burning bright,
In the forests of the night;
What immortal hand or eye,
Could frame thy fearful symmetry?

In what distant deeps or skies.
Burnt the fire of thine eyes?
On what wings dare he aspire?
What the hand, dare seize the fire?

And what shoulder, and what art,
Could twist the sinews of thy heart?
And when thy heart began to beat,
What dread hand? and what dread feet?

What the hammer? what the chain,
In what furnace was thy brain?
What the anvil? what dread grasp,
Dare its deadly terrors clasp!

When the stars threw down their spears
And water’d heaven with their tears:
Did he smile his work to see?
Did he who made the Lamb make thee?

Tiger Tiger burning bright,
In the forests of the night:
What immortal hand or eye,
Dare frame thy fearful symmetry?
'''

translate_poem(poem)

Model 1
тигр тигр горит ярко
в лес в ночи
что за бессмертный руку или глаз
может обернуться на твою симметрию

глубоко и на небе
обжег в твои глаза
крылья ли он в бою
рука соблазняет огонь

и каким плечом и какое искусство
может сверять у тебя в сердце
и когда сердце начинает биться
что с рук и каков страх

что за цепь
каким был твой мозг
и каков страх
невероятно смертоносные подушки

когда звезд спалил их спирс
и все вместе со слезами
он работал на работу
тот кто сделал ягненок для тебя

тигр тигр горит ярко
в лес в ночи
что за бессмертный руку или глаз
сразись на твою симметрию

Model 2
тигр тигр горит ярко
в сумраке ночи
что бессмертная рука или глаз
могла бы жестокостью святостью милей

в далеких небесах и в небесах
обожгла взор твой взор
на какие крылья смел ему светить
рука не смел держать огонь

и что и что за искусство
вытянул бы из своего сердца
и когда твое сердце начинает биться
что страх из рук и какие испуганные ноги

что молота что цепь
в печь как печь
что длани что пугаю

In [37]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
тигр тигр горит ярко
в лес в ночи
что за бессмертную руку или руку
к твоему девственному языку
---
глубоко и в небесах
огонь в твоих глазах
на крыльях он рискнет
огонь пылает
---
и каким артем
может разделять с твоим сердцем
и когда сердце начинает биться
что за руку и чего ты боишься
---
что за цепкой
какая была вышкой
и каков страх
невероятно смертоносный страх
---
когда звёзды пальнули их грозы
и вода сквозь свои слезы
он работал чтобы он работал
он сделал чтобы он тебя сделал
---
тигр тигр горит ярко
в лес в ночи
что за бессмертную руку или руку
сразись к девственному языку

Model 2
горит тигр с ярким взором
в сумраке ночном
что бессмертная рука или глаз
могла бы жестокостью святостью милей
---
в далеких небесах и в небесах
обожгла огонь в ваших глазах
на какие крылья смел ему светить
что рука может светить
---
и какое плечо и какие искусства
твое сердце снова и снова и снова
и когда твое сердце начинает биться
что пугающий страх и нечего бояться
---
что цепь седла
что в пе

In [39]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
огонь в небо
в лес в сраженье
что за руку или рассеянный
может тянуться в твои шикарные косметики
---
глубоко и на небе
лелеял огнем сраженье
на крыльях он заснет
огонь в огне
---
и какой
может разделять с твоей душой
и когда начнётся твой удар
что за руку и какой страшных преград
---
как муравей
в твоей голове
какой страх в руках
невероятно смертоносные стекла
---
когда звёзды пальнули их грозы
и вода сквозь свои слезы
он работал на работу
тот кто сделал ягненок для тебя
---
огонь в небо
в лес в сраженье
что за руку или рассеянный
не смей тянуться в шикарные косметики

Model 2
тигр тигр горит ярко
в сумраке ночи
что бессмертная рука или очей
могла бы жестокостью святостью милей
---
что в далеких или площадях
обожгла взор твой взгляд
как бы крыльями крыльями стремить его
рука не смел держать огонь
---
и какое плечо и что творенье
я отдал бы от твоего томленья
и когда твое сердце начинает биться вновь
что пугающий страх и что страшная дрожь
---
что молота что цепь
в печь как печ

In [31]:
translate_poem(poem,
               sample_temperature = 0.3,
               stresses_attempts = 10)

Model 1
тигр тигр яркий
в лес в ночи
какой бессмертный руку или глаз
я поставил на твою симметрию

глубоко и в небе
подорвал пламя твоих глаз
крылья ли он в бою
рука может поднять огонь

и каким плечом и какое искусство
доброе сердце
и когда влюблялось в твоё сердце
что с рук и каков страх

а что цепь
что там был твой мозг
каков страх
невероятно смертоносные бедны

когда спалил их спирс
и все вместе со слезами
он работал на работу
кто создал ягненок из тебя

тигр тигр яркий
в лес в ночи
что за бессмертный руку или глаз
в твою прелестную симметрию

Model 2
тигр горит ярко
в сумраке леса среди ночи
что бессмертная рука или глаз
могла бы горстью святостью милей

в далеких небесах и в небесах
сожжённый взор твой взор
на какие крылья смел ему светить
кто смел держать огонь

и что у плеча и что искусство
из твоих опалых сердец
когда твое сердце начинает биться
что пугающий страх и что боится

что молот что цепь
в печь как печь
что дубогля что пугающий страх
рассерди их отчаянные костры

когд

In [14]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
тигр тигр яркий
в лес в ночи
что за бессмертный руку или руку
может тянуться в твою языку
---
и в небесах
пылал огонь в твоих глазах
на крыльях он рискнет
рука огонь пылает
---
и каким артем
может быть с твоим сердцем
и когда сердце начинает биться
чего ты боишься
---
что цепкой
с твоей усыпанной вышкой
каков страх
невероятно смертоносный страх
---
когда пальнули их грозы
и залегнут в свои слезы
он работал чтобы он работал
что он тебя сделал
---
тигр тигр яркий
в лес в ночи
что за руку или руку
смещать тебя к страшному языку

Model 2
тигр тигр обжигает ярким взором
в сумраке ночном
что бессмертная рука или глаз
и милей
---
в далеких небесах и в небесах
сожгло пламя в ваших глазах
на какие крылья смел ему светить
что рука пламя светить
---
и что у плеча и какие искусства
твое сердце снова и снова и снова
и когда твоё сердце начинает биться
какой ужас и нечего бояться
---
что цепь седла
в печь печь печь жужжала
что из мрачных страстей
огонь своих гибельных страстей
---
когда звёз

In [15]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
тигр в небо
в лесу в сраженье
что за руку или рассеянный
строки из твоей косметики
---
на самом месте и на небе
огнем огнем сраженье
на крыльях он заснет
что за власть в огне
---
и какой
кружит с твоей душой
и когда в сердце в твой удар
что с рук и что страшных преград
---
как муравей
в твоей голове
какой страх в руках
невероятно смертоносные стекла
---
когда пальнули их грозы
и вода сквозь свои слезы
он работал чтобы посмотреть
кто создал ягненок для тебя
---
тигр огонь в небо
в лес в движенье
глаз рассеянный
примений светлых косметики

Model 2
тигр тигр горит ярко
в лесах среди ночи
что бессмертная рука или очей
могла бы горстью святостью милей
---
что в далеких или площадях
жгло жметный твой взгляд
с крыльями стремить его
что рука высоко держать огонь
---
и какое плечо и что творенье
заяви сердца изнемленья
когда твое сердце начинает биться вновь
что со страхом и что страшная дрожь
---
что молот что цепь
в печь как печь
что по-гульни что какое страданье
сразить его хищные ра

In [16]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 100)

Model 1
тигр тигр яркий
в лес в ночи
смерть за руку или руку
по твоему девственному языку
---
и в небесах
жгли огонь в твоих глазах
на крыльях он рискнет
она пылает
---
и каким артем
с твоим сердцем
и когда сердце начинает биться
и страх и чего ты боишься
---
что цепкой
с какой клейкой вышкой
каков страх
невероятно смертоносный страх
---
когда звезды поднимала им грозы
сквозь свои слезы
он работал чтобы он работал
что он тебя сделал
---
тигр тигр яркий
в лес в ночи
за руку или руку
к твоему страшному языку

Model 2
горит горный тигр с ярким взором
в сумраке ночном
что бессмертная рука или глаз
мне и милей
---
что далекой правды и в небесах
сожгло пламя в ваших глазах
в какие крылья смел ему светить
что рука умеет светить
---
и что с плеча и какие искусства
снова и снова
когда сердце начинает биться
мне страшно и нечего бояться
---
что цепь седла
в печь печь печь жужжала
чепль что из мрачных страстей
огонь своих гибельных страстей
---
когда звёзды сорвали с их весом
и воды к небесам с и

In [17]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 100)

Model 1
смотреть в огонь в небо
в лес в сраженье
бессмертный руку или рассеянный
в мир твоей косметики
---
в глубине и на небе
жгли в сраженье
на крыльях он заснет
что за власть в огне
---
и какой
с твоей душой
и в сердце на твой удар
и страх а до чего преград
---
как муравей
в твоей голове
какой страх в руках
ее долбанные стекла
---
когда пальнули их грозы
и к небесам в свои слезы
он работал на работу
кто создал ягненок для вас
---
тигр в небо
в лес в доложенье
бессмертный руку и рассеянный
посмею тебе страшной косметики

Model 2
тигр тигр горит ярко
в листьях ночи
что бессмертная рука или очей
я мог бы страстям святостью милей
---
в своих далеких или площадях
обожгла взор твой взгляд
заставляет ли крылья стремить его
какая рука и смел держать огонь
---
и какое плечо и что творенье
отдал бы от твоих устремленья
когда твое сердце начинает биться вновь
что со страхом и что страшная дрожь
---
что цепь
в печь как печь
что гульни что грозное страданье
гряди смертельной ядерные раны
---
ког

In [18]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 1000)

Model 1
тигр тигр яркий
в лес в ночь
двойную руку или руку
к твоему девственному языку
---
и в небесах
пламя в твоих глазах
он рискнет
рука огонь пылает
---
и каким артем
с твоим сердцем
и когда сердце начинает биться
страх и то чего ты боишься
---
что цепкой
с вышкой
каков страх
это смертоносный страх
---
когда пальнули их грозы
сквозь свои слезы
он работал
что он у тебя сделал
---
тигр тигр яркий
в лес в ночь
какой бессмертный руку или руку
к грубому языку

Model 2
тигр тигр обжигает ярким взором
в сумраке ночном
что бессмертная рука или глаз
и милей
---
столь глубоких или в небесах
горестный огонь в ваших глазах
в какие крылья смел ему светить
какая рука рука будет светить
---
и за плечо и какие искусства
снова и снова
и когда сердце начинает биться
боюсь и страх и нечего бояться
---
что цепь седла
в печь печь печь жужжала
чепль что из мрачных страстей
рассерди всех своих смертных страстей
---
когда звёзды сорвали с их весом
и воды к небесам с их соколом
он работал чтобы он работал


In [ ]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 1000)

In [40]:
# Fragment from "Весна" by Сергей Есенин, writted in 1924

'''
И выйдет девушка к тебе,
Водой окатит из колодца,
Чтобы в суровом октябре
Ты мог с метелями бороться.

А ночью
Выплывет луна.
Ее не слопали собаки:
Она была лишь не видна
Из-за людской
Кровавой драки.
'''

poem = '''
And the girl will come to you,
Water will be poured from the well,
To in severe October
You could have fought the blizzards.

And at night
The moon will rise.
It wasn't eaten by dogs:
She was only not visible
Because of human
Bloody fight.
'''

translate_poem(poem)

Model 1
и девушка придет к тебе
вода гудит из колодца
в серьезного октября
ты мог бороться с венами

и по ночам
восходит луна
не мои псы
больше ничего не видно
из-за человека
чертову драку

Model 2
и девушка придет к тебе
вылетит вода из колодца
в суровый в октябре
ты могла бы с пулями

и по ночам
взойдет луна
её не было у собаки
в ней едва заметный вид
из-за человека
кровавые бои

Model 3
и девушка близится к тебе
вода вылетит из колодца
средь тягостного октября
ты мог бороться с этими срами

и по ночам
взойдет луна
это не были псы
только она не была
из-за человека
кровавый бой



In [42]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
и девчонка к тебе придет
вода тяжелеет
в серьезное октябрьском
ты мог бороться с медом
---
и по ночам
восходит луна
он не ел с собаками
она была собственными глазами
из-за человека
чертова драка

Model 2
и девушка придет к тебе
вылетит вода из колодца
к тяжелым прискорбным лесам
сражался бы по вражеским горам
---
и ночь напролет
луна взойдет
это не была собака
она была невидимка
из-за человека
кровавая драка

Model 3
и к тебе явится девушка
вода окольется от ручейка
в тяжелым ноябрьском
ты мог бы бороться с костром
---
и в ночь напролет
луна взойдет
была не была собака
она была только не гадалка
из-за человечества
кровавая битва



In [43]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0)

Model 1
и девушка придет с тобой
вода залит вольной волной
в тяжких тубердцев
ты мог сверить
---
и ночью
луна вскочит
не мои псы
больше ничего не видно
из-за человека
чертову драку

Model 2
и девушка придет с тобой
льет вода из волной
к тяжелым прискорбным лесам
сражался бы по вражеским горам
---
и по ночам
взойдет луна
не собаки рисовали
она едва заметала
из-за народа
кровавые бороться

Model 3
и девушка близится к тебе
вода вылетит из побед
в суровой октябрьской
ты мог бороться с костями
---
и в поздний час
луна обложась
псы его не питали
только она не казалась
из-за человека
кровавый бой



In [44]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.SUFFIX,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
и девушка к тебе придет
вода тяжелеет
в октябрьском
ты мог бороться с медом
---
и ночью
восходит луна
его не ел с собаками
она была собственными глазами
из-за человека
чертова драка

Model 2
и девушка придет к тебе
льнет вода из колодца
к тяжким грозным лесам
сражался бы по вражеским горам
---
а ночь напролет
луна взойдет
была не была собака
она была невидимка
из-за человека
кровавая драка

Model 3
и к тебе явится девушка
вода окольется от ручейка
в тяжелым ноябрьском
ты мог бы бороться с костром
---
и в ночь напролет
луна взойдет
она не была собака
была только она не гадалка
из-за человечества
кровавая битва



In [45]:
translate_poem_with_rhyme(poem,
                          rhyme_type = RhymeType.WORD,
                          rhyme_test_counts = (20, 2),
                          max_total_rhyme_tests = 0,
                          sample_temperature = 0.3,
                          stresses_attempts = 10)

Model 1
и девушка придет с тобой
воды с водой наполненной волной
в тяжких тубердцев
можно сверить
---
и ночью
луна вскочит
не мои псы
ничего не видно
из-за людей
в драку

Model 2
и девушка придет с тобой
вылетит вода из волной
к тяжким темнейшим лесам
сражался бы по вражеским горам
---
и ночью
взойдет луна
не собаки рисовали
она не заметала
из-за народа
кровавые бороться

Model 3
и девушка близится к тебе
вода окольется от побед
к суровой октябрьской
ты мог бы бороться с костями
---
в поздний час
луна обложась
его не питали
только она не казалась
потому что из-за человека
кровавый бой

